# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# List all databases to confirm the creation of 'uk_food'
db_list = mongo.list_database_names()
print(db_list)

['admin', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
collection_list = mongo.uk_food.list_collection_names()
pprint(collection_list)

['establishments']


In [6]:
# review a document in the establishments collection
establishments_collection = mongo.uk_food.establishments
sample_document = establishments_collection.find_one()
pprint(sample_document)

{'AddressLine1': 'West Cottage',
 'AddressLine2': 'South Foreland Lighthouse',
 'AddressLine3': 'The Front',
 'AddressLine4': 'St Margarets Bay',
 'BusinessName': 'Mrs Knotts Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.492467848017,
 'FHRSID': 429991,
 'LocalAuthorityBusinessID': 'PI/000055200',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6HP',
 'RatingDate': '2019-05-02T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65ad890d106979e8a547821d'),
 'geocode': {'latitude': '51.14061', 'longitude': '1.371894'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/429991',
            'rel': 'self'}],
 'meta': {'dataSource': None,
  

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
# New restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [9]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)

In [10]:
# Check that the new restaurant was inserted
if insert_result.acknowledged:
    print(f"New restaurant 'Penang Flavours' successfully inserted. Document ID: {insert_result.inserted_id}")
else:
    print("Failed to insert the new restaurant.")

New restaurant 'Penang Flavours' successfully inserted. Document ID: 65aeb90aeeb8f2a1ed2b4a33


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Specify the fields to return (1 to include, 0 to exclude)
fields_to_return = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

# Perform the query and get the result
business_type_result = establishments.find_one(business_type_query, fields_to_return)

# Print the result
if business_type_result:
    pprint(business_type_result)
else:
    print("No document found for the specified BusinessType.")


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = business_type_result['BusinessTypeID']  # Replace with actual ID retrieved

# find the "Penang Flavours" document
penang_flavours_query = {"BusinessName": "Penang Flavours"}

# Update command to set the BusinessTypeID
update_command = {"$set": {"BusinessTypeID": business_type_id}}

# Perform the update
update_result = establishments.update_one(penang_flavours_query, update_command)

In [13]:
# Confirm that the new restaurant was updated
if update_result.matched_count > 0:
    print(f"Penang Flavours updated with BusinessTypeID: {business_type_id}")
else:
    print("Penang Flavours not found or update not required.")

Penang Flavours updated with BusinessTypeID: 1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [17]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}
dover_count = establishments.count_documents(dover_query)
print(f"Number of establishments in Dover Local Authority: {dover_count}")


Number of establishments in Dover Local Authority: 994


In [18]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many(dover_query)
if delete_result.acknowledged:
    print(f"Number of establishments removed from Dover Local Authority: {delete_result.deleted_count}")
else:
    print("Failed to remove establishments from Dover Local Authority.")


Number of establishments removed from Dover Local Authority: 994


In [19]:
# Check if any remaining documents include Dover
new_dover_count = establishments.count_documents(dover_query)
print(f"Number of establishments in Dover Local Authority after deletion: {new_dover_count}")

Number of establishments in Dover Local Authority after deletion: 0


In [20]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()
if remaining_document:
    print("A document in the collection:")
    pprint(remaining_document)
else:
    print("No documents found in the collection.")

A document in the collection:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65ad890d106979e8a5478500'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'sel

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [22]:
# Change the data type from String to Decimal for longitude and latitude
from pymongo import UpdateOne
from bson.decimal128 import Decimal128

# 1. Convert 'longitude' and 'latitude' from string to decimal numbers using Decimal128
bulk_updates = []
for doc in establishments.find({}, {'_id': 1, 'geocode': 1}):
    updates = {}
    if 'longitude' in doc.get('geocode', {}):
        updates['geocode.longitude'] = Decimal128(doc['geocode']['longitude'])
    if 'latitude' in doc.get('geocode', {}):
        updates['geocode.latitude'] = Decimal128(doc['geocode']['latitude'])
    
    if updates:
        bulk_updates.append(UpdateOne({'_id': doc['_id']}, {'$set': updates}))

# Apply the updates
if bulk_updates:
    result = establishments.bulk_write(bulk_updates)
    print(f"Updated {result.modified_count} documents for longitude and latitude fields.")
else:
    print("No documents to update for longitude and latitude fields.")


Updated 38787 documents for longitude and latitude fields.


Use `update_many` to convert `RatingValue` to integer numbers.

In [25]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [26]:
# Change the data type from String to Integer for RatingValue
bulk_updates = []

# Finding documents where 'RatingValue' is a numeric string
for doc in establishments.find({'RatingValue': {'$regex': '^\d+$'}}, {'_id': 1, 'RatingValue': 1}):
    numeric_rating_value = int(doc['RatingValue'])
    bulk_updates.append(UpdateOne({'_id': doc['_id']}, {'$set': {'RatingValue': numeric_rating_value}}))

# Apply the updates
result = establishments.bulk_write(bulk_updates)
print(f"Updated {result.modified_count} documents for RatingValue field.")

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Muqrrab\AppData\Local\Temp\ipykernel_3664\2752197361.py:5: SyntaxWarning: invalid escape sequence '\d'
  for doc in establishments.find({'RatingValue': {'$regex': '^\d+$'}}, {'_id': 1, 'RatingValue': 1}):


Updated 34694 documents for RatingValue field.


In [27]:
# Check that the coordinates and rating value are now numbers
sample_documents = establishments.find({}, {'geocode': 1, 'RatingValue': 1}).limit(5)

for doc in sample_documents:
    longitude = doc.get('geocode', {}).get('longitude')
    latitude = doc.get('geocode', {}).get('latitude')
    rating_value = doc.get('RatingValue')

    print("Document ID:", doc['_id'])
    print("Longitude Type:", type(longitude))
    print("Latitude Type:", type(latitude))
    print("RatingValue Type:", type(rating_value))
    print()

Document ID: 65ad890d106979e8a5478500
Longitude Type: <class 'bson.decimal128.Decimal128'>
Latitude Type: <class 'bson.decimal128.Decimal128'>
RatingValue Type: <class 'int'>

Document ID: 65ad890d106979e8a5478502
Longitude Type: <class 'bson.decimal128.Decimal128'>
Latitude Type: <class 'bson.decimal128.Decimal128'>
RatingValue Type: <class 'int'>

Document ID: 65ad890d106979e8a5478503
Longitude Type: <class 'bson.decimal128.Decimal128'>
Latitude Type: <class 'bson.decimal128.Decimal128'>
RatingValue Type: <class 'int'>

Document ID: 65ad890d106979e8a5478507
Longitude Type: <class 'bson.decimal128.Decimal128'>
Latitude Type: <class 'bson.decimal128.Decimal128'>
RatingValue Type: <class 'int'>

Document ID: 65ad890d106979e8a5478508
Longitude Type: <class 'bson.decimal128.Decimal128'>
Latitude Type: <class 'bson.decimal128.Decimal128'>
RatingValue Type: <class 'int'>

